In [1]:
import $file.^.Magic

import $file.$

In [2]:
val zipName = "sms+spam+collection.zip"
val datasetUrl = s"https://archive.ics.uci.edu/static/public/228/$zipName"
val outputDir = "data/sms-spam-raw"

zipName: String = "sms+spam+collection.zip"
datasetUrl: String = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
outputDir: String = "data/sms-spam-raw"

In [3]:
Magic.!("curl", "--create-dirs", "-O", "--output-dir", outputDir, datasetUrl)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  198k    0  198k    0     0   153k      0 --:--:--  0:00:01 --:--:--  153k


In [5]:
Magic.!("unzip", "-o", s"$outputDir/$zipName", "-d", outputDir)

Archive:  data/sms-spam-raw/sms+spam+collection.zip
  inflating: data/sms-spam-raw/SMSSpamCollection  
  inflating: data/sms-spam-raw/readme  


In [6]:
import scala.io.Source

val datasetRaw = Source.fromFile(s"$outputDir/SMSSpamCollection").mkString

case class SmsSpamRecord(
  text: String,
  isSpam: Boolean
)

type Dataset = Vector[SmsSpamRecord]

val smsSpamRecords: Dataset = datasetRaw.split("\n").map {
  case s"spam\t$text" => SmsSpamRecord(text, isSpam = true)
  case s"ham\t$text" => SmsSpamRecord(text, isSpam = false)
}.toVector

val (spamRecords, notSpamRecords) = smsSpamRecords.partition(_.isSpam)
println(s"Spam count: ${spamRecords.size}")
println(s"Not spam count: ${notSpamRecords.size}")

Spam count: 747
Not spam count: 4827


import scala.io.Source
datasetRaw: String = """ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461

In [7]:
import scala.collection.mutable
import scala.util.Random

val balancedDataset: Dataset = {

  def sample(records: Vector[SmsSpamRecord], targetSize: Int): Vector[SmsSpamRecord] = {
    val balancedDatasetSpam = mutable.Map[String, SmsSpamRecord]()
    while (balancedDatasetSpam.size < targetSize) {
      val randomRecord = records(Random.nextInt(records.size))
      if (!balancedDatasetSpam.contains(randomRecord.text))
        balancedDatasetSpam += randomRecord.text -> randomRecord
    }
    balancedDatasetSpam.values.toVector
  }

  if (spamRecords.size < notSpamRecords.size)
    spamRecords ++ sample(notSpamRecords, targetSize = spamRecords.size)
  else
    notSpamRecords ++ sample(spamRecords, targetSize = notSpamRecords.size)
}

import scala.collection.mutable
import scala.util.Random
balancedDataset: Dataset = Vector(
  SmsSpamRecord(
    text = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "SIX chances to win CASH! From 100 to 20,000 po

In [8]:
type Training = Dataset
type Validation = Dataset
type Test = Dataset

def randomSplit(dataset: Vector[SmsSpamRecord], trainingFraction: Double, validationFraction: Double): (Training, Validation, Test) = {
  val shuffledDataset = Random.shuffle(dataset)
  val trainingSize = (shuffledDataset.size * trainingFraction).floor.toInt
  val validationSize = (shuffledDataset.size * validationFraction).floor.toInt

  val (training, remainingRecords) = shuffledDataset.splitAt(trainingSize)
  val (validation, test) = remainingRecords.splitAt(validationSize)
  (training, validation, test)
}

val (training, validation, test) = randomSplit(balancedDataset, trainingFraction = 0.7, validationFraction = 0.1) 

defined type Training
defined type Validation
defined type Test
defined function randomSplit
training: Training = Vector(
  SmsSpamRecord(
    text = "Ur HMV Quiz cash-balance is currently £500 - to maximize ur cash-in now send HMV1 to 86688 only 150p/msg",
    isSpam = true
  ),
  SmsSpamRecord(text = "Went fast asleep dear.take care.", isSpam = false),
  SmsSpamRecord(
    text = "Dear Subscriber ur draw 4 £100 gift voucher will b entered on receipt of a correct ans. When was Elvis Presleys Birthday? TXT answer to 80062",
    isSpam = true
  ),
  SmsSpamRecord(
    text = "No shit, but I wasn't that surprised, so I went and spent the evening with that french guy I met in town here and we fooled around a bit but I didn't let him fuck me",
    isSpam = false
  ),
  SmsSpamRecord(
    text = "Someone U know has asked our dating service 2 contact you! Cant Guess who? CALL 09058097189 NOW all will be revealed. POBox 6, LS15HB 150p ",
    isSpam = true
  ),
  SmsSpamRecord(text = "I wait 4

In [9]:
import $ivy.`com.github.tototoshi::scala-csv:2.0.0`

import scala.util.Using
import com.github.tototoshi.csv.CSVWriter

val textHeader = "Text"
val labelHeader = "Label"

def writeToCsv(path: String, dataset: Dataset): Unit = {
  val headers = Vector(textHeader, labelHeader)

  Using.resource(CSVWriter.open(path)) { writer =>
    val rows = dataset.map {
      case SmsSpamRecord(text, isSpam) => Vector(text, if (isSpam) "1" else "0")
    }
    writer.writeAll(headers +: rows)
  }
}

val trainingCsv = "data/training.csv"
writeToCsv(trainingCsv, training)
val validationCsv = "data/validation.csv"
writeToCsv(validationCsv, validation)
val testCsv = "data/test.csv"
writeToCsv(testCsv, test)

import $ivy.$
import scala.util.Using
import com.github.tototoshi.csv.CSVWriter
textHeader: String = "Text"
labelHeader: String = "Label"
defined function writeToCsv
trainingCsv: String = "data/training.csv"
validationCsv: String = "data/validation.csv"
testCsv: String = "data/test.csv"

In [10]:
Magic.!("pip", "install", "tiktoken==0.7.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import $ivy.`dev.scalapy::scalapy-core:0.5.3`

import me.shadaj.scalapy.py
import py.SeqConverters

val tiktoken = py.module("tiktoken")

val tokenizer = tiktoken.get_encoding("gpt2")
val endOfTextToken = "<|endoftext|>"
val encodedEndOfTextToken = tokenizer.encode(endOfTextToken, allowed_special = py.Dynamic.global.set(Seq(endOfTextToken).toPythonProxy))
println(encodedEndOfTextToken)

[50256]


import $ivy.$
import me.shadaj.scalapy.py
import py.SeqConverters
tiktoken: py.Module = <module 'tiktoken' from '/usr/local/lib/python3.12/site-packages/tiktoken/__init__.py'>
tokenizer: py.Dynamic = <Encoding 'gpt2'>
endOfTextToken: String = "<|endoftext|>"
encodedEndOfTextToken: py.Dynamic = [50256]

In [12]:
Magic.!("pip", "install", "torch==2.4.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import com.github.tototoshi.csv.CSVReader
import py.PyQuote

val torch = py.module("torch")

type Tokenizer = py.Dynamic

// Workaround to define a class that inherits from a Python class
py.exec {
  s"""from torch.utils.data import Dataset
     |
     |class SpamDataset(Dataset):
     |  def __init__(self, init):
     |    init(self)
     |
     |  def __getitem__(self, index):
     |    return self.getItem(index)
     |  
     |  def __len__(self):
     |    return self.len()
     |""".stripMargin
}
def SpamDataset(
  csvPath: String,
  tokenizer: Tokenizer,
  maxLength: Option[Int] = None,
  paddingTokenId: Int = 50_256
): py.Dynamic = {
  val smsSpamRecords = Using.resource(CSVReader.open(csvPath)) { csvReader =>
    csvReader.iteratorWithHeaders.map { row =>
      SmsSpamRecord(text = row(textHeader), isSpam = row(labelHeader).toInt > 0)
    }.toVector
  }
  val encodedTexts = {
    val encodedTexts = smsSpamRecords.map(_.text).map(tokenizer.encode(_).as[Seq[Int]].toVector)
    val padToLength = maxLength.getOrElse(encodedTexts.map(_.length).max)
    encodedTexts.map(_.padTo(padToLength, paddingTokenId))
  }
    
  val init = (self: py.Dynamic) => {
    self.maxLength = encodedTexts.head.length
    
    val getItem = (index: Int) => {
      val textTensor = torch.tensor(encodedTexts(index).toPythonProxy, dtype = torch.long)
      val labelTensor = torch.tensor(if (smsSpamRecords(index).isSpam) 1 else 0, dtype = torch.long)
      (textTensor, labelTensor)
    }
    self.getItem = getItem

    val len = () => smsSpamRecords.size
    self.len = len
  }
  py.Dynamic.global.SpamDataset(init)
}

import com.github.tototoshi.csv.CSVReader
import py.PyQuote
torch: py.Module = <module 'torch' from '/usr/local/lib/python3.12/site-packages/torch/__init__.py'>
defined type Tokenizer
defined function SpamDataset

In [14]:
val trainingDataset = SpamDataset(trainingCsv, tokenizer)
val validationDataset = SpamDataset(validationCsv, tokenizer, maxLength = Some(trainingDataset.maxLength.as[Int]))
val testDataset = SpamDataset(testCsv, tokenizer, maxLength = Some(trainingDataset.maxLength.as[Int]))

trainingDataset: py.Dynamic = <SpamDataset object at 0xffff4671be30>
validationDataset: py.Dynamic = <SpamDataset object at 0xffff5403c200>
testDataset: py.Dynamic = <SpamDataset object at 0xffff54096420>

In [15]:
val batchSize = 8
torch.manual_seed(123)

val trainingDataLoader = torch.utils.data.DataLoader(
  dataset = trainingDataset, 
  batch_size = batchSize,
  shuffle = true,
  num_workers = 0,
  drop_last = true
)
val validationDataLoader = torch.utils.data.DataLoader(
  dataset = validationDataset, 
  batch_size = batchSize,
  num_workers = 0,
  drop_last = false
)
val testDataLoader = torch.utils.data.DataLoader(
  dataset = testDataset, 
  batch_size = batchSize,
  num_workers = 0,
  drop_last = false
)

println(s"${py.Dynamic.global.len(trainingDataLoader)} training batches")
println(s"${py.Dynamic.global.len(validationDataLoader)} validation batches")
println(s"${py.Dynamic.global.len(testDataLoader)} test batches")

130 training batches
19 validation batches
38 test batches


batchSize: Int = 8
res15_1: py.Dynamic = <torch._C.Generator object at 0xffff6072c770>
trainingDataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xffff46719460>
validationDataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xffff46b8bd10>
testDataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xffff60b75400>

In [16]:
case class GPTConfig(
  vocabularySize: Int,
  contextLength: Int,
  embeddingDimension: Int,
  attentionHeadsCount: Int,
  layersCount: Int,
  dropoutRate: Double,
  queryKeyValueBias: Boolean
)

defined class GPTConfig

In [17]:
type TorchTensor = py.Dynamic

// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class MultiHeadAttention(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def MultiHeadAttention(
  inputDimension: Int,
  outputDimension: Int,
  dropoutProbability: Double,
  contextLength: Int,
  headsCount: Int,
  queryKeyValueBias: Boolean
): py.Dynamic = {
  assert(outputDimension % headsCount == 0, "Output dimension must be a multiple of heads count")
  val headDimension = outputDimension / headsCount
    
  val init = (self: py.Dynamic) => {
    self.weightsQuery = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
    self.weightsKey = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
    self.weightsValue = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
    self.outputProjection = torch.nn.Linear(outputDimension, outputDimension)
    self.dropout = torch.nn.Dropout(dropoutProbability)
    self.register_buffer("mask", torch.triu(torch.ones(contextLength, contextLength), diagonal = 1))
      
    val forward = (batchedInputs: TorchTensor) => {
      val (batchesCount, tokensCount, tokenDimension) = batchedInputs.shape.as[(Int, Int, Int)]
      val queries = self.weightsQuery(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val keys = self.weightsKey(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val values = self.weightsValue(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val attentionScores = py"$queries @ $keys.transpose(2, 3)"
      attentionScores.masked_fill_(py"${self.mask}.bool()[:$tokensCount, :$tokensCount]", -torch.inf)
      val attentionWeights = self.dropout(torch.softmax(py"$attentionScores / $headDimension**0.5", dim = -1))
      self.outputProjection(
        py"$attentionWeights @ $values"
          .transpose(1, 2)
          .reshape(batchesCount, tokensCount, outputDimension)
      )
    }
    self.forward = forward
  }
  py.Dynamic.global.MultiHeadAttention(init)
}

defined type TorchTensor
defined function MultiHeadAttention

In [18]:
// Workaround to define a class that inherits from a Python class
// Because it mostly uses Python operators, it's implemented fully in Python
py.exec {
  s"""import torch
     |import torch.nn as nn
     |
     |class GELU(nn.Module):
     |  def __init__(self):
     |    super().__init__()
     |
     |  def forward(self, inputs):
     |    return 0.5 * inputs * (
     |      1 + torch.tanh(
     |        torch.sqrt(torch.tensor(2.0 / torch.pi)) * (inputs + 0.044715 * torch.pow(inputs, 3))
     |      )
     |    )
     |""".stripMargin
}
def GELU() = py.Dynamic.global.GELU()

defined function GELU

In [19]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class FeedForward(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def FeedForward(
  embeddingDimension: Int
): py.Dynamic = {
  val init = (self: py.Dynamic) => {
    self.layers = torch.nn.Sequential(
      torch.nn.Linear(embeddingDimension, 4 * embeddingDimension),
      GELU(),
      torch.nn.Linear(4 * embeddingDimension, embeddingDimension)
    )
      
    val forward = (inputs: TorchTensor) => self.layers(inputs)
    self.forward = forward
  }
  py.Dynamic.global.FeedForward(init)
}

defined function FeedForward

In [20]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class NormalizationLayer(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def NormalizationLayer(
  embeddingDimension: Int
): py.Dynamic = {
  val epsilon = 1e-5
  val init = (self: py.Dynamic) => {
    self.scale = torch.nn.Parameter(torch.ones(embeddingDimension))
    self.shift = torch.nn.Parameter(torch.zeros(embeddingDimension))
      
    val forward = (inputs: TorchTensor) => {
      val mean = inputs.mean(dim = -1, keepdim = true)
      val variance = inputs.`var`(dim = -1, keepdim = true, unbiased = false)
      val normalizedInputs = py"($inputs - $mean) / torch.sqrt($variance + $epsilon)"
      py"${self.scale} * $normalizedInputs + ${self.shift}"
    }
    self.forward = forward
  }
  py.Dynamic.global.NormalizationLayer(init)
}

defined function NormalizationLayer

In [21]:
import scala.util.chaining._

py.exec {
  s"""import torch.nn as nn
     |
     |class TransformerBlock(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def TransformerBlock(
  config: GPTConfig
): py.Dynamic = {
  val init = (self: py.Dynamic) => {
    self.multiHeadAttention = MultiHeadAttention(
      inputDimension = config.embeddingDimension,
      outputDimension = config.embeddingDimension,
      dropoutProbability = config.dropoutRate,
      contextLength = config.contextLength,
      headsCount = config.attentionHeadsCount,
      queryKeyValueBias = config.queryKeyValueBias
    )
    self.feedForward = FeedForward(config.embeddingDimension)
    self.normalization1 = NormalizationLayer(config.embeddingDimension)
    self.normalization2 = NormalizationLayer(config.embeddingDimension)
    self.dropoutShortcut = torch.nn.Dropout(config.dropoutRate)
    
    val forward = (inputs: TorchTensor) => {
      val shortcut = inputs
      val newShortcut = inputs
        .pipe(self.normalization1(_))
        .pipe(self.multiHeadAttention(_))
        .pipe(self.dropoutShortcut(_))
        .pipe(o => py"$o + $shortcut")
      newShortcut
        .pipe(self.normalization2(_))
        .pipe(self.feedForward(_))
        .pipe(self.dropoutShortcut(_))
        .pipe(o => py"$o + $newShortcut")
    }
    self.forward = forward
  }
  py.Dynamic.global.TransformerBlock(init)
}

import scala.util.chaining._
defined function TransformerBlock

In [22]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class GPTModel(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
type Model = py.Dynamic
def GPTModel(
  config: GPTConfig
): Model = {
  val transformerBlocks = Seq.fill(config.layersCount)(TransformerBlock(config))
  val init = (self: py.Dynamic) => {
    self.tokenEmbeddingLayer = torch.nn.Embedding(config.vocabularySize, config.embeddingDimension)
    self.positionEmbeddingLayer = torch.nn.Embedding(config.contextLength, config.embeddingDimension)
    self.dropoutEmbeddingLayer = torch.nn.Dropout(config.dropoutRate)
    self.transformerBlocksLayer = py"nn.Sequential(*${transformerBlocks.toPythonProxy})"
    self.finalNormalizationLayer = NormalizationLayer(config.embeddingDimension)
    self.outputLayer = torch.nn.Linear(config.embeddingDimension, config.vocabularySize, bias = false)
      
    val forward = (batchedInputs: TorchTensor) => {
      val (_, sequenceLength) = batchedInputs.shape.as[(Int, Int)]
      val tokenEmbeddings = self.tokenEmbeddingLayer(batchedInputs)
      val positionEmbeddings = self.positionEmbeddingLayer(torch.arange(sequenceLength, device = batchedInputs.device))
      py"$tokenEmbeddings + $positionEmbeddings"
        .pipe(self.dropoutEmbeddingLayer(_))
        .pipe(self.transformerBlocksLayer(_))
        .pipe(self.finalNormalizationLayer(_))
        .pipe(self.outputLayer(_))
    }
    self.forward = forward
  }
  py.Dynamic.global.GPTModel(init)
}

defined type Model
defined function GPTModel

In [23]:
val baseUrl = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2/124M" // backup
// val baseUrl = "https://openaipublic.blob.core.windows.net/gpt-2/models/124M" // backup
val hparamsFilename = "hparams.json"
val filenames = List("checkpoint", "encoder.json", hparamsFilename, "model.ckpt.data-00000-of-00001", "model.ckpt.index", "model.ckpt.meta", "vocab.bpe")

val outputDir = "data/openai124M"

baseUrl: String = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2/124M"
hparamsFilename: String = "hparams.json"
filenames: List[String] = List(
  "checkpoint",
  "encoder.json",
  "hparams.json",
  "model.ckpt.data-00000-of-00001",
  "model.ckpt.index",
  "model.ckpt.meta",
  "vocab.bpe"
)
outputDir: String = "data/openai124M"

In [24]:
filenames.foreach { filename =>
  println(s"Downloading $filename...")
  Magic.!("curl", "--create-dirs", "-O", "--output-dir", outputDir, s"$baseUrl/$filename")
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    77  100    77    0     0     74      0  0:00:01  0:00:01 --:--:--    74


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  5 1017k    5 56906    0     0  50551      0  0:00:20  0:00:01  0:00:19 50538
100 1017k  100 1017k    0     0   543k      0  0:00:01  0:00:01 --:--:--  543k


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    90  100    90    0     0    134      0 --:--:-- --:--:-- --:--:--   134


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  474M    0  391k    0     0   253k      0  0:31:55  0:00:01  0:31:54  253k
  2  474M    2 10.3M    0     0  4083k      0  0:01:59  0:00:02  0:01:57 4082k
  5  474M    5 25.1M    0     0  7176k      0  0:01:07  0:00:03  0:01:04 7175k
  8  474M    8 40.2M    0     0  8982k      0  0:00:54  0:00:04  0:00:50 8982k
 11  474M   11 56.5M    0     0  10.1M      0  0:00:46  0:00:05  0:00:41 11.3M
 15  474M   15 71.4M    0     0  10.9M      0  0:00:43  0:00:06  0:00:37 14.2M
 18  474M   18 86.1M    0     0  11.4M      0  0:00:41  0:00:07  0:00:34 15.3M
 21  474M   21  103M    0     0  12.0M      0  0:00:39  0:00:08  0:00:31 15.6M
 25  474M   25  119M    0     0  12.4M      0  0:00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5215  100  5215    0     0   7723      0 --:--:-- --:--:-- --:--:--  7714
100  5215  100  5215    0     0   7721      0 --:--:-- --:--:-- --:--:--  7714


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  8  460k    8 40512    0     0  42148      0  0:00:11 --:--:--  0:00:11 42112
100  460k  100  460k    0     0   290k      0  0:00:01  0:00:01 --:--:--  290k


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 46  445k   46  207k    0     0   152k      0  0:00:02  0:00:01  0:00:01  152k
100  445k  100  445k    0     0   285k      0  0:00:01  0:00:01 --:--:--  285k


In [25]:
Magic.!("pip", "install", "tensorflow==2.16.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [26]:
import $ivy.`com.lihaoyi::ujson:4.1.0`

import scala.io.Source

val hparamsMap = ujson.read(Source.fromFile(s"$outputDir/$hparamsFilename").mkString)

val gptConfig = GPTConfig(
  vocabularySize = hparamsMap("n_vocab").num.toInt,
  contextLength = hparamsMap("n_ctx").num.toInt,
  embeddingDimension = hparamsMap("n_embd").num.toInt,
  attentionHeadsCount = hparamsMap("n_head").num.toInt,
  layersCount = hparamsMap("n_layer").num.toInt,
  dropoutRate = 0.1,
  queryKeyValueBias = true
)

import $ivy.$
import scala.io.Source
hparamsMap: ujson.Value.Value = Obj(
  value = Map(
    "n_vocab" -> Num(value = 50257.0),
    "n_ctx" -> Num(value = 1024.0),
    "n_embd" -> Num(value = 768.0),
    "n_head" -> Num(value = 12.0),
    "n_layer" -> Num(value = 12.0)
  )
)
gptConfig: GPTConfig = GPTConfig(
  vocabularySize = 50257,
  contextLength = 1024,
  embeddingDimension = 768,
  attentionHeadsCount = 12,
  layersCount = 12,
  dropoutRate = 0.1,
  queryKeyValueBias = true
)

In [27]:
val tf = py.module("tensorflow")
val np = py.module("numpy")

tf: py.Module = <module 'tensorflow' from '/usr/local/lib/python3.12/site-packages/tensorflow/__init__.py'>
np: py.Module = <module 'numpy' from '/usr/local/lib/python3.12/site-packages/numpy/__init__.py'>

In [28]:
val checkpoint = tf.train.latest_checkpoint(outputDir)
val variableNames = tf.train.list_variables(checkpoint).as[Seq[(String, Seq[Int])]].map { 
  case (variableName, _) => variableName 
}.toList
variableNames.sorted.foreach(println)

model/h0/attn/c_attn/b
model/h0/attn/c_attn/w
model/h0/attn/c_proj/b
model/h0/attn/c_proj/w
model/h0/ln_1/b
model/h0/ln_1/g
model/h0/ln_2/b
model/h0/ln_2/g
model/h0/mlp/c_fc/b
model/h0/mlp/c_fc/w
model/h0/mlp/c_proj/b
model/h0/mlp/c_proj/w
model/h1/attn/c_attn/b
model/h1/attn/c_attn/w
model/h1/attn/c_proj/b
model/h1/attn/c_proj/w
model/h1/ln_1/b
model/h1/ln_1/g
model/h1/ln_2/b
model/h1/ln_2/g
model/h1/mlp/c_fc/b
model/h1/mlp/c_fc/w
model/h1/mlp/c_proj/b
model/h1/mlp/c_proj/w
model/h10/attn/c_attn/b
model/h10/attn/c_attn/w
model/h10/attn/c_proj/b
model/h10/attn/c_proj/w
model/h10/ln_1/b
model/h10/ln_1/g
model/h10/ln_2/b
model/h10/ln_2/g
model/h10/mlp/c_fc/b
model/h10/mlp/c_fc/w
model/h10/mlp/c_proj/b
model/h10/mlp/c_proj/w
model/h11/attn/c_attn/b
model/h11/attn/c_attn/w
model/h11/attn/c_proj/b
model/h11/attn/c_proj/w
model/h11/ln_1/b
model/h11/ln_1/g
model/h11/ln_2/b
model/h11/ln_2/g
model/h11/mlp/c_fc/b
model/h11/mlp/c_fc/w
model/h11/mlp/c_proj/b
model/h11/mlp/c_proj/w
model/h2/attn/c_

checkpoint: py.Dynamic = data/openai124M/model.ckpt
variableNames: List[String] = List(
  "model/h0/attn/c_attn/b",
  "model/h0/attn/c_attn/w",
  "model/h0/attn/c_proj/b",
  "model/h0/attn/c_proj/w",
  "model/h0/ln_1/b",
  "model/h0/ln_1/g",
  "model/h0/ln_2/b",
  "model/h0/ln_2/g",
  "model/h0/mlp/c_fc/b",
  "model/h0/mlp/c_fc/w",
  "model/h0/mlp/c_proj/b",
  "model/h0/mlp/c_proj/w",
  "model/h1/attn/c_attn/b",
  "model/h1/attn/c_attn/w",
  "model/h1/attn/c_proj/b",
  "model/h1/attn/c_proj/w",
  "model/h1/ln_1/b",
  "model/h1/ln_1/g",
  "model/h1/ln_2/b",
  "model/h1/ln_2/g",
  "model/h1/mlp/c_fc/b",
  "model/h1/mlp/c_fc/w",
  "model/h1/mlp/c_proj/b",
  "model/h1/mlp/c_proj/w",
  "model/h10/attn/c_attn/b",
  "model/h10/attn/c_attn/w",
  "model/h10/attn/c_proj/b",
  "model/h10/attn/c_proj/w",
  "model/h10/ln_1/b",
  "model/h10/ln_1/g",
  "model/h10/ln_2/b",
  "model/h10/ln_2/g",
  "model/h10/mlp/c_fc/b",
  "model/h10/mlp/c_fc/w",
  "model/h10/mlp/c_proj/b",
  "model/h10/mlp/c_proj/w",


In [29]:
type NpArray = py.Dynamic

def toTorchParameter(npArray: NpArray) =
  torch.nn.Parameter(torch.tensor(npArray))

def loadModelWeights(model: Model): Unit =
  variableNames.foreach { variableName =>
    val variableValue = np.squeeze(tf.train.load_variable(checkpoint, variableName))
    variableName.split("/").drop(1).toList match {
      case s"h$transformerBlockIndexString" :: tail =>
        val transformerBlockIndex = transformerBlockIndexString.toInt
        tail match {
          case "attn" :: tail =>
            val multiHeadAttention = model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention
            tail match {
              case "c_attn" :: tail =>
                val Seq(queryVariableValue, keyVariableValue, valueVariableValue) = np.split(variableValue, 3, axis = -1).as[Seq[NpArray]]
                tail match {
                  case "b" :: _ => 
                    multiHeadAttention.weightsQuery.bias = toTorchParameter(queryVariableValue)
                    multiHeadAttention.weightsKey.bias = toTorchParameter(keyVariableValue)
                    multiHeadAttention.weightsValue.bias = toTorchParameter(valueVariableValue)
                  case "w" :: _ => 
                    multiHeadAttention.weightsQuery.weight = toTorchParameter(queryVariableValue.T)
                    multiHeadAttention.weightsKey.weight = toTorchParameter(keyVariableValue.T)
                    multiHeadAttention.weightsValue.weight = toTorchParameter(valueVariableValue.T)
                }
              case "c_proj" :: tail =>
                tail match {
                  case "b" :: _ => multiHeadAttention.outputProjection.bias = toTorchParameter(variableValue)
                  case "w" :: _ => multiHeadAttention.outputProjection.weight = toTorchParameter(variableValue.T)
                }
            }
          case "ln_1" :: tail =>
            val normalization1 = model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).normalization1
            val torchParameter = toTorchParameter(variableValue)
            tail match {
              case "b" :: _ => normalization1.shift = torchParameter
              case "g" :: _ => normalization1.scale = torchParameter
            }
          case "ln_2" :: tail =>
            val normalization2 = model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).normalization2
            val torchParameter = toTorchParameter(variableValue)
            tail match {
              case "b" :: _ => normalization2.shift = torchParameter
              case "g" :: _ => normalization2.scale = torchParameter
            }
          case "mlp" :: tail =>
            val feedForward = model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).feedForward
            tail match {
              case "c_fc" :: tail =>
                val layer0 = feedForward.layers.bracketAccess(0)
                tail match {
                  case "b" :: _ => layer0.bias = toTorchParameter(variableValue)
                  case "w" :: _ => layer0.weight = toTorchParameter(variableValue.T)
                }
              case "c_proj" :: tail =>
                val layer2 = feedForward.layers.bracketAccess(2)
                tail match {
                  case "b" :: _ => layer2.bias = toTorchParameter(variableValue)
                  case "w" :: _ => layer2.weight = toTorchParameter(variableValue.T)
                }
            }
        }
      case "ln_f" :: tail =>
        val finalNormalizationLayer = model.finalNormalizationLayer
        val torchParameter = toTorchParameter(variableValue)
        tail match {
          case "b" :: _ => finalNormalizationLayer.shift = torchParameter
          case "g" :: _ => finalNormalizationLayer.scale = torchParameter
        }
      case "wpe" :: _ => model.positionEmbeddingLayer.weight = toTorchParameter(variableValue)
      case "wte" :: _ => 
        val torchParameter = toTorchParameter(variableValue)
        model.tokenEmbeddingLayer.weight = torchParameter
        model.outputLayer.weight = torchParameter
    }
  }

cmd29.sc:18: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "w"))), Nil
                tail match {
                ^
cmd29.sc:29: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "w"))), Nil
                tail match {
                ^
cmd29.sc:15: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("c_attn", "c_proj"))), Nil
            tail match {
            ^
cmd29.sc:37: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "g"))), Nil
            tail match {
            ^
cmd29.sc:44: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "g"))), Nil
            tail match {
            ^
cmd29.sc:53: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b",

defined type NpArray
defined function toTorchParameter
defined function loadModelWeights

In [30]:
val model = GPTModel(gptConfig)
loadModelWeights(model)
val device = torch.device(if (torch.cuda.is_available().as[Boolean]) "cuda" else "cpu")
model.to(device)
model.eval()

model: Model = GPTModel(
  (tokenEmbeddingLayer): Embedding(50257, 768)
  (positionEmbeddingLayer): Embedding(1024, 768)
  (dropoutEmbeddingLayer): Dropout(p=0.1, inplace=False)
  (transformerBlocksLayer): Sequential(
    (0): TransformerBlock(
      (multiHeadAttention): MultiHeadAttention(
        (weightsQuery): Linear(in_features=768, out_features=768, bias=True)
        (weightsKey): Linear(in_features=768, out_features=768, bias=True)
        (weightsValue): Linear(in_features=768, out_features=768, bias=True)
        (outputProjection): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feedForward): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (normalization1): NormalizationLayer()
      (normalization2): NormalizationLayer()
      (d

In [31]:
def textToTokenIds(
  text: String, 
  tokenizer: Tokenizer
): TorchTensor = {
  val allowedSpecial = py.Dynamic.global.set(Seq("<|endoftext|>").toPythonProxy)
  val encodedText = tokenizer.encode(text, allowed_special = allowedSpecial)
  torch.tensor(encodedText).unsqueeze(0)
}
    
def tokenIdsToText(
  tokenIds: TorchTensor, 
  tokenizer: Tokenizer
): String =
  tokenizer.decode(tokenIds.squeeze(0).tolist()).as[String]

def generateTextSimple(
  model: Model,
  maxNewTokens: Int,
  contextLength: Int
)(
  encodedInput: TorchTensor
): TorchTensor =
  LazyList.iterate(encodedInput) { currentEncodedOutput =>
    val croppedInput = py"$currentEncodedOutput[:, -$contextLength:]"
    val logits = py.`with`(torch.no_grad()) { _ =>
      model(croppedInput)
    }
    py"$logits[:, -1, :]"
      .pipe(torch.softmax(_, dim = -1))
      .pipe(torch.argmax(_, dim = -1, keepdim = true))
      .pipe(nextEncodedOutput => torch.cat((currentEncodedOutput, nextEncodedOutput), dim = 1))
  }.drop(maxNewTokens).head

defined function textToTokenIds
defined function tokenIdsToText
defined function generateTextSimple

In [32]:
val exampleText = "Every effort moves you"
val encodedText = textToTokenIds(exampleText, tokenizer)
val encodedTextOutput = generateTextSimple(model, maxNewTokens = 15, contextLength = gptConfig.contextLength)(encodedText)
val decodedTextOutput = tokenIdsToText(encodedTextOutput, tokenizer)
println(decodedTextOutput)

Every effort moves you forward.

The first step is to understand the importance of your work


exampleText: String = "Every effort moves you"
encodedText: TorchTensor = tensor([[6109, 3626, 6100,  345]])
encodedTextOutput: TorchTensor = tensor([[6109, 3626, 6100,  345, 2651,   13,  198,  198,  464,  717, 2239,  318,
          284, 1833,  262, 6817,  286,  534,  670]])
decodedTextOutput: String = """Every effort moves you forward.

The first step is to understand the importance of your work"""